In [ ]:
import cudf

data=cudf.read_csv("/home/pavit21178/Nalin_OFF/Data/OFF_FNDDS_COLS_NOVA.csv",index_col=0)
# keep only data w 1,2,3,4 in the novaclass column
data = data[data['novaclass'].isin([1,2,3,4])]







In [ ]:
# use random forest to predict the NOVA group
from cuml.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from cuml.metrics import accuracy_score

X = data.drop('novaclass', axis=1)
y=data['novaclass']



In [ ]:
from cuml.preprocessing import SimpleImputer
import cupy as cp
imp_mean = SimpleImputer(missing_values=cp.nan, strategy='mean')


In [ ]:
from cuml.preprocessing import StandardScaler


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X_train=imp_mean.fit_transform(X_train)
X_test=imp_mean.transform(X_test)
rf=RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred=rf.predict(X_test)



In [ ]:
print(accuracy_score(y_test,y_pred))
y_test_np=y_test.to_numpy()
y_pred_np=y_pred.to_numpy()
from sklearn.metrics import classification_report
print(classification_report(y_test_np, y_pred_np))

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [10, 20, 30, None],
    'max_features': ['sqrt', 'log2',1.0],
    'min_samples_split': [2, 10, 20],
    'min_samples_leaf': [1, 4, 10],
    'bootstrap': [True, False]
}


In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Set up the RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_grid, n_iter=10, cv=5, random_state=42, verbose=2)

# Fit the RandomizedSearchCV on the imputed training data
random_search.fit(X_train.to_numpy(), y_train.to_numpy())


In [ ]:
print("Best parameters found: ", random_search.best_params_)
print("Best cross-validation score: ", random_search.best_score_)

# Evaluate the best model on the imputed test set
best_model = random_search.best_estimator_
test_score = best_model.score(X_test, y_test)
print("Test set score of the best model: ", test_score)

In [ ]:

rf=RandomForestClassifier(n_estimators=500,max_depth=30,max_features=1.0,min_samples_split=20,min_samples_leaf=1,bootstrap=True)

In [ ]:
import numpy as np
import pandas as pd
from cuml.ensemble import RandomForestClassifier as cuRF
from cuml.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, matthews_corrcoef, precision_recall_fscore_support,
    classification_report, confusion_matrix
)
from imblearn.over_sampling import SMOTE
import os
import joblib
import cudf
import cupy as cp

def evaluate_classifier_with_stratified_smote(X_train, y_train, X_test, y_test, classifier, num_folds=10, save_path=None, model_name=None):
    k_fold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=0)

    accuracies_val = []
    accuracies_train = []
    train_scores = []
    val_scores = []
    train_mcc = []
    val_mcc = []
    train_f1 = []
    val_f1 = []
    train_precision = []
    val_precision = []
    train_recall = []
    val_recall = []

    for fold_idx, (train_indices, val_indices) in enumerate(k_fold.split(X_train, y_train), 1):
        X_fold_train, X_fold_val = X_train.iloc[train_indices].to_pandas(), X_train.iloc[val_indices].to_pandas()
        y_fold_train, y_fold_val = y_train.iloc[train_indices].to_pandas(), y_train.iloc[val_indices].to_pandas()

        smote = SMOTE()
        X_fold_train_resampled, y_fold_train_resampled = smote.fit_resample(X_fold_train, y_fold_train)

        X_fold_train_resampled = cudf.DataFrame.from_pandas(X_fold_train_resampled)
        y_fold_train_resampled = cudf.Series.from_pandas(y_fold_train_resampled)
        X_fold_val = cudf.DataFrame.from_pandas(X_fold_val)
        y_fold_val = cudf.Series.from_pandas(y_fold_val)

        classifier.fit(X_fold_train_resampled, y_fold_train_resampled)
        train_score = accuracy_score(y_fold_train_resampled.to_pandas(), classifier.predict(X_fold_train_resampled).to_pandas())
        train_scores.append(train_score)
        y_train_pred = classifier.predict(X_fold_train_resampled).to_pandas()
        fold_accuracy_train = accuracy_score(y_fold_train_resampled.to_pandas(), np.round(y_train_pred))
        accuracies_train.append(fold_accuracy_train)

        train_mcc.append(matthews_corrcoef(y_fold_train_resampled.to_pandas(), np.round(y_train_pred)))
        train_f1.append(precision_recall_fscore_support(y_fold_train_resampled.to_pandas(), np.round(y_train_pred), average='weighted')[2])
        precision, recall, _, _ = precision_recall_fscore_support(y_fold_train_resampled.to_pandas(), np.round(y_train_pred), average='weighted')
        train_precision.append(precision)
        train_recall.append(recall)

        val_score = accuracy_score(y_fold_val.to_pandas(), classifier.predict(X_fold_val).to_pandas())
        val_scores.append(val_score)

        y_val_pred = classifier.predict(X_fold_val).to_pandas()
        fold_accuracy_val = accuracy_score(y_fold_val.to_pandas(), np.round(y_val_pred))
        accuracies_val.append(fold_accuracy_val)

        # Additional metrics for validation set: MCC, F1 Score, Precision, Recall
        val_mcc.append(matthews_corrcoef(y_fold_val.to_pandas(), np.round(y_val_pred)))
        val_f1.append(precision_recall_fscore_support(y_fold_val.to_pandas(), np.round(y_val_pred), average='weighted')[2])
        precision, recall, _, _ = precision_recall_fscore_support(y_fold_val.to_pandas(), np.round(y_val_pred), average='weighted')
        val_precision.append(precision)
        val_recall.append(recall)

    average_accuracy_val = sum(accuracies_val) / num_folds
    print(f'Average Accuracy Val K-Fold: {average_accuracy_val * 100:.4f}')

    average_accuracy_train = sum(accuracies_train) / num_folds
    print(f'Average Accuracy Train K-Fold: {average_accuracy_train * 100:.4f}')

    average_mcc_val = sum(val_mcc) / num_folds
    print(f'Average MCC Val K-Fold: {average_mcc_val:.4f}')

    average_mcc_train = sum(train_mcc) / num_folds
    print(f'Average MCC Train K-Fold: {average_mcc_train:.4f}')

    average_f1_val = sum(val_f1) / num_folds
    print(f'Average F1 Score Val K-Fold: {average_f1_val:.4f}')

    average_f1_train = sum(train_f1) / num_folds
    print(f'Average F1 Score Train K-Fold: {average_f1_train:.4f}')

    average_precision_val = sum(val_precision) / num_folds
    print(f'Average Precision Val K-Fold: {average_precision_val:.4f}')

    average_precision_train = sum(train_precision) / num_folds
    print(f'Average Precision Train K-Fold: {average_precision_train:.4f}')

    average_recall_val = sum(val_recall) / num_folds
    print(f'Average Recall Val K-Fold: {average_recall_val:.4f}')

    average_recall_train = sum(train_recall) / num_folds
    print(f'Average Recall Train K-Fold: {average_recall_train:.4f}')

    X_test = cudf.DataFrame.from_pandas(X_test)
    y_test = cudf.Series.from_pandas(y_test)
    y_pred = classifier.predict(X_test).to_pandas()

    accuracy = accuracy_score(y_test.to_pandas(), y_pred)
    print(f'Accuracy Test: {accuracy * 100:.2f}%')

    test_mcc = matthews_corrcoef(y_test.to_pandas(), np.round(y_pred))
    print(f'MCC Test: {test_mcc:.4f}')

    test_f1 = precision_recall_fscore_support(y_test.to_pandas(), np.round(y_pred), average='weighted')[2]
    print(f'F1 Score Test: {test_f1:.4f}')

    test_precision, test_recall, _, _ = precision_recall_fscore_support(y_test.to_pandas(), np.round(y_pred), average='weighted')
    print(f'Precision Test: {test_precision:.4f}')
    print(f'Recall Test: {test_recall:.4f}')

    class_report = classification_report(y_test.to_pandas(), y_pred)
    print('Classification Report:')
    print(class_report)

    conf_matrix_test = confusion_matrix(y_test.to_pandas(), np.round(y_pred))
    print('Confusion Matrix Test:')
    print(conf_matrix_test)

    if save_path and model_name:
        output_filename = os.path.join(save_path, f"{model_name}_outputs.txt")
        with open(output_filename, 'w') as f:
            f.write(f'Average Accuracy Val K-Fold: {average_accuracy_val * 100:.4f}\n')
            f.write(f'Average Accuracy Train K-Fold: {average_accuracy_train * 100:.4f}\n')
            f.write(f'Average MCC Val K-Fold: {average_mcc_val:.4f}\n')
            f.write(f'Average MCC Train K-Fold: {average_mcc_train:.4f}\n')
            f.write(f'Average F1 Score Val K-Fold: {average_f1_val:.4f}\n')
            f.write(f'Average F1 Score Train K-Fold: {average_f1_train:.4f}\n')
            f.write(f'Average Precision Val K-Fold: {average_precision_val:.4f}\n')
            f.write(f'Average Precision Train K-Fold: {average_precision_train:.4f}\n')
            f.write(f'Average Recall Val K-Fold: {average_recall_val:.4f}\n')
            f.write(f'Average Recall Train K-Fold: {average_recall_train:.4f}\n')
            f.write(f'Accuracy Test: {accuracy * 100:.2f}%\n')
            f.write(f'MCC Test: {test_mcc:.4f}\n')
            f.write(f'F1 Score Test: {test_f1:.4f}\n')
            f.write(f'Precision Test: {test_precision:.4f}\n')
            f.write(f'Recall Test: {test_recall:.4f}\n')
            f.write('Classification Report:\n')
            f.write(class_report + '\n')
            f.write('Confusion Matrix Test:\n')
            f.write(str(conf_matrix_test) + '\n')
        print(f"Outputs saved to: {output_filename}")
        # save the model using joblib
        joblib.dump(classifier, os.path.join(save_path, f"{model_name}.joblib"))


In [ ]:
evaluate_classifier_with_stratified_smote(X_train, y_train, X_test, y_test, rf, num_folds=10, save_path="/home/pavit21178/Nalin_OFF/Models_44/RF", model_name="RF_strat_Kfold_Smote")

In [ ]:
import numpy as np
import pandas as pd
from cuml.ensemble import RandomForestClassifier as cuRF
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, matthews_corrcoef, precision_recall_fscore_support,
    classification_report, confusion_matrix
)
from imblearn.over_sampling import SMOTE
import os
import joblib
import cudf
import cupy as cp

def evaluate_classifier_with_kfold_smote(X_train, y_train, X_test, y_test, classifier, num_folds=10, save_path=None, model_name=None):
    k_fold = KFold(n_splits=num_folds, shuffle=True, random_state=0)

    accuracies_val = []
    accuracies_train = []
    train_scores = []
    val_scores = []
    train_mcc = []
    val_mcc = []
    train_f1 = []
    val_f1 = []
    train_precision = []
    val_precision = []
    train_recall = []
    val_recall = []

    for fold_idx, (train_indices, val_indices) in enumerate(k_fold.split(X_train, y_train), 1):
        X_fold_train, X_fold_val = X_train.iloc[train_indices].to_pandas(), X_train.iloc[val_indices].to_pandas()
        y_fold_train, y_fold_val = y_train.iloc[train_indices].to_pandas(), y_train.iloc[val_indices].to_pandas()

        smote = SMOTE()
        X_fold_train_resampled, y_fold_train_resampled = smote.fit_resample(X_fold_train, y_fold_train)

        X_fold_train_resampled = cudf.DataFrame.from_pandas(X_fold_train_resampled)
        y_fold_train_resampled = cudf.Series.from_pandas(y_fold_train_resampled)
        X_fold_val = cudf.DataFrame.from_pandas(X_fold_val)
        y_fold_val = cudf.Series.from_pandas(y_fold_val)

        classifier.fit(X_fold_train_resampled, y_fold_train_resampled)
        train_score = accuracy_score(y_fold_train_resampled.to_pandas(), classifier.predict(X_fold_train_resampled).to_pandas())
        train_scores.append(train_score)
        y_train_pred = classifier.predict(X_fold_train_resampled).to_pandas()
        fold_accuracy_train = accuracy_score(y_fold_train_resampled.to_pandas(), np.round(y_train_pred))
        accuracies_train.append(fold_accuracy_train)

        train_mcc.append(matthews_corrcoef(y_fold_train_resampled.to_pandas(), np.round(y_train_pred)))
        train_f1.append(precision_recall_fscore_support(y_fold_train_resampled.to_pandas(), np.round(y_train_pred), average='weighted')[2])
        precision, recall, _, _ = precision_recall_fscore_support(y_fold_train_resampled.to_pandas(), np.round(y_train_pred), average='weighted')
        train_precision.append(precision)
        train_recall.append(recall)

        val_score = accuracy_score(y_fold_val.to_pandas(), classifier.predict(X_fold_val).to_pandas())
        val_scores.append(val_score)

        y_val_pred = classifier.predict(X_fold_val).to_pandas()
        fold_accuracy_val = accuracy_score(y_fold_val.to_pandas(), np.round(y_val_pred))
        accuracies_val.append(fold_accuracy_val)

        # Additional metrics for validation set: MCC, F1 Score, Precision, Recall
        val_mcc.append(matthews_corrcoef(y_fold_val.to_pandas(), np.round(y_val_pred)))
        val_f1.append(precision_recall_fscore_support(y_fold_val.to_pandas(), np.round(y_val_pred), average='weighted')[2])
        precision, recall, _, _ = precision_recall_fscore_support(y_fold_val.to_pandas(), np.round(y_val_pred), average='weighted')
        val_precision.append(precision)
        val_recall.append(recall)

    average_accuracy_val = sum(accuracies_val) / num_folds
    print(f'Average Accuracy Val K-Fold: {average_accuracy_val * 100:.4f}')

    average_accuracy_train = sum(accuracies_train) / num_folds
    print(f'Average Accuracy Train K-Fold: {average_accuracy_train * 100:.4f}')

    average_mcc_val = sum(val_mcc) / num_folds
    print(f'Average MCC Val K-Fold: {average_mcc_val:.4f}')

    average_mcc_train = sum(train_mcc) / num_folds
    print(f'Average MCC Train K-Fold: {average_mcc_train:.4f}')

    average_f1_val = sum(val_f1) / num_folds
    print(f'Average F1 Score Val K-Fold: {average_f1_val:.4f}')

    average_f1_train = sum(train_f1) / num_folds
    print(f'Average F1 Score Train K-Fold: {average_f1_train:.4f}')

    average_precision_val = sum(val_precision) / num_folds
    print(f'Average Precision Val K-Fold: {average_precision_val:.4f}')

    average_precision_train = sum(train_precision) / num_folds
    print(f'Average Precision Train K-Fold: {average_precision_train:.4f}')

    average_recall_val = sum(val_recall) / num_folds
    print(f'Average Recall Val K-Fold: {average_recall_val:.4f}')

    average_recall_train = sum(train_recall) / num_folds
    print(f'Average Recall Train K-Fold: {average_recall_train:.4f}')

    X_test = cudf.DataFrame.from_pandas(X_test)
    y_test = cudf.Series.from_pandas(y_test)
    y_pred = classifier.predict(X_test).to_pandas()

    accuracy = accuracy_score(y_test.to_pandas(), y_pred)
    print(f'Accuracy Test: {accuracy * 100:.2f}%')

    test_mcc = matthews_corrcoef(y_test.to_pandas(), np.round(y_pred))
    print(f'MCC Test: {test_mcc:.4f}')

    test_f1 = precision_recall_fscore_support(y_test.to_pandas(), np.round(y_pred), average='weighted')[2]
    print(f'F1 Score Test: {test_f1:.4f}')

    test_precision, test_recall, _, _ = precision_recall_fscore_support(y_test.to_pandas(), np.round(y_pred), average='weighted')
    print(f'Precision Test: {test_precision:.4f}')
    print(f'Recall Test: {test_recall:.4f}')

    class_report = classification_report(y_test.to_pandas(), y_pred)
    print('Classification Report:')
    print(class_report)

    conf_matrix_test = confusion_matrix(y_test.to_pandas(), np.round(y_pred))
    print('Confusion Matrix Test:')
    print(conf_matrix_test)

    if save_path and model_name:
        output_filename = os.path.join(save_path, f"{model_name}_outputs.txt")
        with open(output_filename, 'w') as f:
            f.write(f'Average Accuracy Val K-Fold: {average_accuracy_val * 100:.4f}\n')
            f.write(f'Average Accuracy Train K-Fold: {average_accuracy_train * 100:.4f}\n')
            f.write(f'Average MCC Val K-Fold: {average_mcc_val:.4f}\n')
            f.write(f'Average MCC Train K-Fold: {average_mcc_train:.4f}\n')
            f.write(f'Average F1 Score Val K-Fold: {average_f1_val:.4f}\n')
            f.write(f'Average F1 Score Train K-Fold: {average_f1_train:.4f}\n')
            f.write(f'Average Precision Val K-Fold: {average_precision_val:.4f}\n')
            f.write(f'Average Precision Train K-Fold: {average_precision_train:.4f}\n')
            f.write(f'Average Recall Val K-Fold: {average_recall_val:.4f}\n')
            f.write(f'Average Recall Train K-Fold: {average_recall_train:.4f}\n')
            f.write(f'Accuracy Test: {accuracy * 100:.2f}%\n')
            f.write(f'MCC Test: {test_mcc:.4f}\n')
            f.write(f'F1 Score Test: {test_f1:.4f}\n')
            f.write(f'Precision Test: {test_precision:.4f}\n')
            f.write(f'Recall Test: {test_recall:.4f}\n')
            f.write('Classification Report:\n')
            f.write(class_report + '\n')
            f.write('Confusion Matrix Test:\n')
            f.write(str(conf_matrix_test) + '\n')
        print(f"Outputs saved to: {output_filename}")
        # save the model using joblib
        joblib.dump(classifier, os.path.join(save_path, f"{model_name}.joblib"))


In [ ]:
evaluate_classifier_with_kfold_smote(X_train, y_train, X_test, y_test, rf, num_folds=10, save_path="/home/pavit21178/Nalin_OFF/Models_44/RF", model_name="RF_Kfold_Smote")

In [ ]:
import numpy as np
import pandas as pd
from cuml.ensemble import RandomForestClassifier as cuRF
from cuml.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, matthews_corrcoef, precision_recall_fscore_support,
    classification_report, confusion_matrix
)
from imblearn.over_sampling import SMOTE
import os
import joblib
import cudf
import cupy as cp

def evaluate_classifier_with_stratified(X_train, y_train, X_test, y_test, classifier, num_folds=10, save_path=None, model_name=None):
    k_fold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=0)

    accuracies_val = []
    accuracies_train = []
    train_scores = []
    val_scores = []
    train_mcc = []
    val_mcc = []
    train_f1 = []
    val_f1 = []
    train_precision = []
    val_precision = []
    train_recall = []
    val_recall = []

    for fold_idx, (train_indices, val_indices) in enumerate(k_fold.split(X_train, y_train), 1):
        X_fold_train, X_fold_val = X_train.iloc[train_indices].to_pandas(), X_train.iloc[val_indices].to_pandas()
        y_fold_train, y_fold_val = y_train.iloc[train_indices].to_pandas(), y_train.iloc[val_indices].to_pandas()

        
        X_fold_train_resampled, y_fold_train_resampled = (X_fold_train, y_fold_train)

        X_fold_train_resampled = cudf.DataFrame.from_pandas(X_fold_train_resampled)
        y_fold_train_resampled = cudf.Series.from_pandas(y_fold_train_resampled)
        X_fold_val = cudf.DataFrame.from_pandas(X_fold_val)
        y_fold_val = cudf.Series.from_pandas(y_fold_val)

        classifier.fit(X_fold_train_resampled, y_fold_train_resampled)
        train_score = accuracy_score(y_fold_train_resampled.to_pandas(), classifier.predict(X_fold_train_resampled).to_pandas())
        train_scores.append(train_score)
        y_train_pred = classifier.predict(X_fold_train_resampled).to_pandas()
        fold_accuracy_train = accuracy_score(y_fold_train_resampled.to_pandas(), np.round(y_train_pred))
        accuracies_train.append(fold_accuracy_train)

        train_mcc.append(matthews_corrcoef(y_fold_train_resampled.to_pandas(), np.round(y_train_pred)))
        train_f1.append(precision_recall_fscore_support(y_fold_train_resampled.to_pandas(), np.round(y_train_pred), average='weighted')[2])
        precision, recall, _, _ = precision_recall_fscore_support(y_fold_train_resampled.to_pandas(), np.round(y_train_pred), average='weighted')
        train_precision.append(precision)
        train_recall.append(recall)

        val_score = accuracy_score(y_fold_val.to_pandas(), classifier.predict(X_fold_val).to_pandas())
        val_scores.append(val_score)

        y_val_pred = classifier.predict(X_fold_val).to_pandas()
        fold_accuracy_val = accuracy_score(y_fold_val.to_pandas(), np.round(y_val_pred))
        accuracies_val.append(fold_accuracy_val)

        # Additional metrics for validation set: MCC, F1 Score, Precision, Recall
        val_mcc.append(matthews_corrcoef(y_fold_val.to_pandas(), np.round(y_val_pred)))
        val_f1.append(precision_recall_fscore_support(y_fold_val.to_pandas(), np.round(y_val_pred), average='weighted')[2])
        precision, recall, _, _ = precision_recall_fscore_support(y_fold_val.to_pandas(), np.round(y_val_pred), average='weighted')
        val_precision.append(precision)
        val_recall.append(recall)

    average_accuracy_val = sum(accuracies_val) / num_folds
    print(f'Average Accuracy Val K-Fold: {average_accuracy_val * 100:.4f}')

    average_accuracy_train = sum(accuracies_train) / num_folds
    print(f'Average Accuracy Train K-Fold: {average_accuracy_train * 100:.4f}')

    average_mcc_val = sum(val_mcc) / num_folds
    print(f'Average MCC Val K-Fold: {average_mcc_val:.4f}')

    average_mcc_train = sum(train_mcc) / num_folds
    print(f'Average MCC Train K-Fold: {average_mcc_train:.4f}')

    average_f1_val = sum(val_f1) / num_folds
    print(f'Average F1 Score Val K-Fold: {average_f1_val:.4f}')

    average_f1_train = sum(train_f1) / num_folds
    print(f'Average F1 Score Train K-Fold: {average_f1_train:.4f}')

    average_precision_val = sum(val_precision) / num_folds
    print(f'Average Precision Val K-Fold: {average_precision_val:.4f}')

    average_precision_train = sum(train_precision) / num_folds
    print(f'Average Precision Train K-Fold: {average_precision_train:.4f}')

    average_recall_val = sum(val_recall) / num_folds
    print(f'Average Recall Val K-Fold: {average_recall_val:.4f}')

    average_recall_train = sum(train_recall) / num_folds
    print(f'Average Recall Train K-Fold: {average_recall_train:.4f}')

    X_test = cudf.DataFrame.from_pandas(X_test)
    y_test = cudf.Series.from_pandas(y_test)
    y_pred = classifier.predict(X_test).to_pandas()

    accuracy = accuracy_score(y_test.to_pandas(), y_pred)
    print(f'Accuracy Test: {accuracy * 100:.2f}%')

    test_mcc = matthews_corrcoef(y_test.to_pandas(), np.round(y_pred))
    print(f'MCC Test: {test_mcc:.4f}')

    test_f1 = precision_recall_fscore_support(y_test.to_pandas(), np.round(y_pred), average='weighted')[2]
    print(f'F1 Score Test: {test_f1:.4f}')

    test_precision, test_recall, _, _ = precision_recall_fscore_support(y_test.to_pandas(), np.round(y_pred), average='weighted')
    print(f'Precision Test: {test_precision:.4f}')
    print(f'Recall Test: {test_recall:.4f}')

    class_report = classification_report(y_test.to_pandas(), y_pred)
    print('Classification Report:')
    print(class_report)

    conf_matrix_test = confusion_matrix(y_test.to_pandas(), np.round(y_pred))
    print('Confusion Matrix Test:')
    print(conf_matrix_test)

    if save_path and model_name:
        output_filename = os.path.join(save_path, f"{model_name}_outputs.txt")
        with open(output_filename, 'w') as f:
            f.write(f'Average Accuracy Val K-Fold: {average_accuracy_val * 100:.4f}\n')
            f.write(f'Average Accuracy Train K-Fold: {average_accuracy_train * 100:.4f}\n')
            f.write(f'Average MCC Val K-Fold: {average_mcc_val:.4f}\n')
            f.write(f'Average MCC Train K-Fold: {average_mcc_train:.4f}\n')
            f.write(f'Average F1 Score Val K-Fold: {average_f1_val:.4f}\n')
            f.write(f'Average F1 Score Train K-Fold: {average_f1_train:.4f}\n')
            f.write(f'Average Precision Val K-Fold: {average_precision_val:.4f}\n')
            f.write(f'Average Precision Train K-Fold: {average_precision_train:.4f}\n')
            f.write(f'Average Recall Val K-Fold: {average_recall_val:.4f}\n')
            f.write(f'Average Recall Train K-Fold: {average_recall_train:.4f}\n')
            f.write(f'Accuracy Test: {accuracy * 100:.2f}%\n')
            f.write(f'MCC Test: {test_mcc:.4f}\n')
            f.write(f'F1 Score Test: {test_f1:.4f}\n')
            f.write(f'Precision Test: {test_precision:.4f}\n')
            f.write(f'Recall Test: {test_recall:.4f}\n')
            f.write('Classification Report:\n')
            f.write(class_report + '\n')
            f.write('Confusion Matrix Test:\n')
            f.write(str(conf_matrix_test) + '\n')
        print(f"Outputs saved to: {output_filename}")
        # save the model using joblib
        joblib.dump(classifier, os.path.join(save_path, f"{model_name}.joblib"))


In [ ]:
evaluate_classifier_with_stratified(X_train, y_train, X_test, y_test, rf, num_folds=10, save_path="/home/pavit21178/Nalin_OFF/Models_44/RF", model_name="RF_strat")